In [56]:
# !pip install pyomo
# !pip install idaes-pse

In [57]:

import numpy as np
import pandas as pd
import time
# from contextlib import redirect_stdout

from pyomo.environ import *

solvername = 'ipopt'


Caso não encontre o solver na pasta:
- Importar arquivos de extensões da biblioteca IDAES;
- Copiar o arquivo do solver ipopt para a pasta do projeto e excluir os outros arquivos.

In [58]:
# search for solver executable in the PATH

import os

if os.path.exists('bin\\' + solvername + '.exe'):
    print(solvername + ' executable found')
    # add current folder to the PATH
    os.environ['PATH'] += ';.'

else:
    print(solvername + ' executable not found')

    # ------------------------- # 

    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

    # ------------------------- # 

    # import shutil

    # # file names
    # src_file = "bin\\" + solvername + ".exe"
    # dst_file = solvername + ".exe"

    # shutil.copyfile(src_file, dst_file)
    # print('Copied')

    # # ------------------------- #

    # # excluir pasta bin
    # import os, shutil

    # pasta = './bin'
    # shutil.rmtree(pasta)
    # print('Deleted')

    # ------------------------- #


ipopt executable found


In [59]:

sig = 0

UTEs = [('UTE1', 'UTE2', 'UTE3', 'UTE4'),
        (1000, 970, 700, 680),
        (16.19, 17.26, 16.6, 16.5),
        (0.00048, 0.00031, 0.002, 0.00211),
        (455, 455, 130, 130),
        (150, 150, 20, 20)]
UTEs = np.transpose(UTEs)
UTE = pd.DataFrame(UTEs, columns=['Nome','a','b','c','PGmax','PGmin'])



Dem = [('Hora1', 450, 45),
       ('Hora2', 530, 53),
       ('Hora3', 600, 60),
       ('Hora4', 540, 54),
       ('Hora5', 400, 40),
       ('Hora6', 280, 28),
       ('Hora7', 290, 29),
       ('Hora8', 500, 50)]
Dem = pd.DataFrame(Dem, columns=['Hora','Dem', 'Res'])

UTE['a'] = UTE['a'].astype('float')
UTE['b'] = UTE['b'].astype('float')
UTE['c'] = UTE['c'].astype('float')
UTE['PGmax'] = UTE['PGmax'].astype('float')
UTE['PGmin'] = UTE['PGmin'].astype('float')


In [60]:

def Otimiza_UC(UTE,Dem,Sig,Print):
    #=====================================Modelagem do Problema============================================================
    modelo = ConcreteModel('Otimizacao Unnit Commitment')

    def limite_UTE(modelo,pg,ext):
        return (UTE[UTE['Nome'] == pg]['PGmin'].values[0], UTE[UTE['Nome'] == pg]['PGmax'].values[0])
    def limite_U(modelo,pg,ext):
        return (0, 1)
    def limite_X(modelo,pg,ext):
        return (0.0, 20.0)

    # ----------------    Declarar Variáveis
    modelo.Pg = Var(UTE['Nome'], Dem['Hora'], bounds=limite_UTE, domain=NonNegativeReals)
    modelo.U = Var(UTE['Nome'], Dem['Hora'], bounds=limite_U, domain=NonNegativeReals) # Função Decisão de Operação - FDO

    if Sig == 1:
        modelo.X = Var(UTE['Nome'], Dem['Hora'], bounds=limite_X, domain=NonNegativeReals) # Argumento da FDO  -  utilizado na aproximação sigmoidal


    # ----------------    Declarar Função Objetivo
    def objective_rule(modelo):
        fob = 0
        for h in Dem['Hora']:
            fob += sum((UTE[UTE['Nome'] == g]['a'].values[0] +
                        UTE[UTE['Nome'] == g]['b'].values[0]*modelo.Pg[g,h] +
                        UTE[UTE['Nome'] == g]['c'].values[0]*modelo.Pg[g,h]**2)*modelo.U[g,h] for g in UTE['Nome'])
        return fob


    modelo.objetivo = Objective(rule=objective_rule,sense=minimize)

    # ----------------     Declarar Restrições
    modelo.AtendDem = ConstraintList()
    modelo.AtendDemRes = ConstraintList()
    modelo.LimGer = ConstraintList()
    # modelo.Sigmoide = ConstraintList()


    for h in Dem['Hora']:
        modelo.AtendDem.add(sum(modelo.U[g,h]*modelo.Pg[g,h] for g in UTE['Nome']) == Dem[Dem['Hora'] == h]['Dem'].values[0])
    for h in Dem['Hora']:
        modelo.AtendDemRes.add(sum(modelo.U[g,h]*UTE[UTE['Nome'] == g]['PGmax'].values[0] for g in UTE['Nome']) >=
                               Dem[Dem['Hora'] == h]['Dem'].values[0] + Dem[Dem['Hora'] == h]['Res'].values[0])
    # for h in Dem['Hora']:
    #     for g in UTE['Nome']:
    #         modelo.LimGer.add(UTE[UTE['Nome'] == g]['PGmin'].values[0] <= modelo.Pg[g,h])
    #         modelo.LimGer.add(modelo.Pg[g,h] <= UTE[UTE['Nome'] == g]['PGmax'].values[0])



    # FDO Sigmoidal - definição

    if Sig == 1:
        modelo.FDO_Sigmoide = ConstraintList() # Restrição da FDO Sigmoidal
        alpha=2
        for h in Dem['Hora']:
            for g in UTE['Nome']:
                modelo.FDO_Sigmoide.add(modelo.U[g,h]  -  (exp(alpha * (modelo.X[g,h])) - 1) / (exp(alpha * (modelo.X[g,h])) + 1) == 0)


    # caso Sig == 0, o algoritmo decidirá o valor de U entre 0 e 1; logo, será feita uma aproximação linear




    #------------------------- Adição da aproximação pela função sigmoide
    # for h in Dem['Hora']:
    #     for g in UTE['Nome']:
    #         modelo.Sigmoide.add(0 <= 1/(1+ exp(-modelo.U[g,h])))
    #         modelo.Sigmoide.add(1/(1+ exp(-modelo.U[g,h])) <= 1)


    # if sig == 0:
    #     modelo.Sigmoide.deactivate()



    # with open('resumo_modelo.txt', 'w') as f:
    #     # Redirecione a saída para o arquivo
    #     with redirect_stdout(f):
    #         # Imprima um resumo do modelo
    #         modelo.pprint()

    # modelo.pprint()



    ### Executar a Otimização.
    # solver = SolverFactory('ipopt')
    solver = SolverFactory('ipopt',executable='bin/ipopt')
    # solver = SolverFactory('ipopt',executable='ipopt.exe')

    resultados_seq = solver.solve(modelo, tee=Print)


    # resultados_seq = solver.solve(model_seq, tee=False)
    # resultados_con = solver.solve(model_con, tee=False)

    # Relatório dos resultados de otimização

    print('Status Final do Problema de Otimização:', resultados_seq.solver.status)
    print('Condição de Término:', resultados_seq.solver.termination_condition)
    print('Resultado Função Objetivo:', value(modelo.objetivo))


    # ------------------------- #

    u_val = modelo.U.extract_values()

    df_u = pd.DataFrame(columns=UTE['Nome'], index=Dem['Hora'])
    for i in u_val:
        df_u[i[0]][i[1]] = u_val[i]


    # ordenar, para cada hora, as UTEs em ordem decrescente de U
    ordem_despacho = np.zeros((len(Dem['Hora']), len(UTE['Nome'])))
    for i in range(len(Dem['Hora'])):
        # guardar o número da UTE 
        ordem_despacho[i] = np.argsort(df_u.iloc[i])[::-1]

    # print('Ordem de Despacho das UTEs:')
    # print(ordem_despacho)


    return modelo, ordem_despacho


In [61]:
t = time.time()

# Modelo com aproximação linear
modelo_lin, ordem_despacho_lin = Otimiza_UC(UTE, Dem, 0, Print=False)

t_lin = time.time() - t
print('Tempo:', t_lin, 'segundos')
print(1*'\n ' + 45*'-' + 2*'\n')


#  --------------------------- #


# Modelo com aproximação sigmoide
modelo_sig, ordem_despacho_sig = Otimiza_UC(UTE, Dem, 1, Print=False)

t_sig = time.time() - t - t_lin
print('Tempo:', t_sig, 'segundos')


Status Final do Problema de Otimização: ok
Condição de Término: optimal
Resultado Função Objetivo: 67863.08835063892
Tempo: 0.19339776039123535 segundos

 ---------------------------------------------


Status Final do Problema de Otimização: ok
Condição de Término: optimal
Resultado Função Objetivo: 67863.08878362423
Tempo: 0.12238359451293945 segundos
